In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from keras.datasets import mnist
import tensorflow.keras as kb
from tensorflow.keras import backend
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer


from plotnine import *

from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score

from sklearn.linear_model import LinearRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler #Z-score variables

from sklearn.model_selection import train_test_split # simple TT split cv
from sklearn.model_selection import KFold # k-fold cv
from sklearn.model_selection import LeaveOneOut #LOO cv


# Basic Keras Compnents

## Model Objects
- `Model()`: an object that groups layers together to be trained and to make predictions


With `Model()` objects we can either use the **Functional API** to interface with them, or we can subclass the `Model()` object. 

### Functional API
Here, we treat layers as functions that have input tensors and output tensors. Each layer takes in the output from the previous layer. 

In [2]:
# Functional API building

inputs = kb.Input(shape = (25,))

x = kb.layers.Dense(10)(inputs)

x = kb.layers.Dense(5)(x)

x = kb.layers.Dense(2)(x)

outputs = kb.layers.Dense(1, activation = tf.nn.sigmoid)(x)

model = kb.Model(inputs = inputs, outputs = outputs)

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 25)]              0         
                                                                 
 dense (Dense)               (None, 10)                260       
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 dense_2 (Dense)             (None, 2)                 12        
                                                                 
 dense_3 (Dense)             (None, 1)                 3         
                                                                 
Total params: 330
Trainable params: 330
Non-trainable params: 0
_________________________________________________________________


Or we can create a new class that inherits from `Model()`. We won't do this too often in this class, but it is useful to know that it exists. 

First we create a class that inherets from `kb.Model`, and then we create an `__init__` method that first calls the superclass' `__init__()` and then defines every layer that we need. We want this to happen in the constructor, otherwise the layers might be created more than once (which we do not want).

Then we create a `call()` method which basically defines what a forward pass of your model looks like. It takes in the default `self` arugment as well as some input to the model. This looks similar to how we defined things using the Functional API. Then we return the output of the model. 

Now we can use this subclass to build a model!
- We create inputs
- We put those inputs into our model object
- We put both into a `Model()` object


In [4]:
# we won't do this often but its nice to know

class MyModel(kb.Model):

    def __init__(self):
        # call init from inhereted class
        super().__init__()

        # create all layers here so they're only created once
        self.layer1 = kb.layers.Dense(10, input_shape = [25])
        self.layer2 = kb.layers.Dense(5)
        self.layer3 = kb.layers.Dense(2)
        self.layer4 = kb.layers.Dense(1, activation = "sigmoid")
    
    def call(self, inputs):
        # define what a forward pass looks like
        x = self.layer1(inputs)
        x = self.layer2(x)
        x = self.layer3(x)
        return(self.layer4(x))

# define input shape
inputs = kb.Input(shape = (25,))

# create a model using your custom class
x = MyModel()(inputs)

# shove the inputs and outputs into a model object
my_model = kb.Model(inputs = inputs, outputs = x)

# show me the model
my_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 25)]              0         
                                                                 
 my_model_1 (MyModel)        (None, 1)                 330       
                                                                 
Total params: 330
Trainable params: 330
Non-trainable params: 0
_________________________________________________________________


## Sequential Object

- `Sequential()`: an object that groups layers together in a linear stack (less flexible than `Model` but typically all we need)

This is what we've done so far (and what we did in CPSC 392). We create a `Sequential()` object and give it a list of layers to add (in order).


In [ ]:
# give Sequential a list of layers
my_model = kb.Sequential([
    kb.layers.Dense(10, input_shape = [25]),
    kb.layers.Dense(5),
    kb.layers.Dense(2),
    kb.layers.Dense(1, activation = "sigmoid")
])

my_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 10)                260       
                                                                 
 dense_13 (Dense)            (None, 5)                 55        
                                                                 
 dense_14 (Dense)            (None, 2)                 12        
                                                                 
 dense_15 (Dense)            (None, 1)                 3         
                                                                 
Total params: 330
Trainable params: 330
Non-trainable params: 0
_________________________________________________________________


If we want to add layers dynamically, we can use `.add()` and `.pop()` to add and pop layers on/off our model. This would be useful, for example, if we wanted to loop through a list of values and add layers with those values.

In [5]:

# use .add() to dynamically add layers
my_model = kb.Sequential()
my_model.add(kb.layers.Dense(10, input_shape = [25]))
my_model.add(kb.layers.Dense(5))
my_model.add(kb.layers.Dense(2))
my_model.add(kb.layers.Dense(1, activation = "sigmoid"))

my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 10)                260       
                                                                 
 dense_13 (Dense)            (None, 5)                 55        
                                                                 
 dense_14 (Dense)            (None, 2)                 12        
                                                                 
 dense_15 (Dense)            (None, 1)                 3         
                                                                 
Total params: 330
Trainable params: 330
Non-trainable params: 0
_________________________________________________________________



## Layers
Keras has many pre-defined layers that we can use (we'll learn about more of them as we learn about more model structures). For now the important layers are:

- `Dense()`: A basic densely connected layer with `units` nodes. Densely connected means that every node in the previous layer is connected to every node in the current layer. 
- `Activation()`: applies an activation function (defined by the `activation` argument) to the values coming into it. This is largely the same as using the `activation` argument in a `Dense` Layer but is useful when you want to do an operation to the layer output BEFORE applying the activation (e.g. `BatchNormalization`)
- `Input()`: A basic layer that defined the input of a model. It tells the model what the initial tensor of data that we expect to come in looks like. The `shape` argument tells the model what a *single sample* of data looks like (not a batch of samples)


`Dense` Layers tend to be the basis of most of our Neural Networks, so let's get to know the documentation a little!

- **Question** look at the [documentation](https://keras.io/api/layers/core_layers/dense/) for `Dense` layers. If I wanted to NOT have a bias for that layer, how might I tell python that?
Make 'use_bias = false'
- **Question** look at the [documentation](https://keras.io/api/layers/core_layers/dense/) for `Dense` layers. If you do not supply a value for `activation` what activation does it use?
If there is no value for activation, no activation is applied.


There are many activation functions (or you can even define your own), let's look at the `activation` documentation and see what's available:

- **Question** look at the [documentation](keras.io/api/layers/activations/) for `activations`. What basic activation functions are available?

The following basic activation funcitons are available: relu, sigmoid, softmax, softplus, softsign, tanh, selu, elu, and exponential.

- Modify the code below to add a `ReLu` activation to the middle layer (either using the `activation` argument in `Dense()` or by adding an `Activation()` layer)



In [ ]:
### YOUR CODE HERE ###
activated_model = kb.Sequential([
    kb.Input(shape = [25]),
    kb.layers.Dense(10), 
    kb.layers.Dense(5, activation = "relu"), # add relu here
    kb.layers.Dense(2),
    kb.layers.Dense(1, activation = "sigmoid")
])

activated_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_78 (Dense)            (None, 10)                260       
                                                                 
 dense_79 (Dense)            (None, 5)                 55        
                                                                 
 dense_80 (Dense)            (None, 2)                 12        
                                                                 
 dense_81 (Dense)            (None, 1)                 3         
                                                                 
Total params: 330
Trainable params: 330
Non-trainable params: 0
_________________________________________________________________


## Try It Out
Build a model with the following structure in 3 different ways:

- input size of 9 
- 2 hidden layers (with 7 and 3 nodes respectively) and `relu` activations
- output layer with 1 node and a sigmoid activation

1. Build the model using a basic `Sequential()` object and using `.add()` to add each layer. Set the activation(s) using an `Activation()` layer. 
2. Build the model using the Functional API method with `Model()`. Set the activation(s) using the `activation` argument in each layer where necessary. 
3. Build the model by subclassing `Model()`. Build all your layers in the `__init__()` method, and define a forward pass using your `call()` method. Then use the class to build your model. Set the activation(s) using the `activation` argument in each layer where necessary. 

In [22]:
# 1. `Sequential()`

# use .add() to dynamically add layers
my_model1 = kb.Sequential()
my_model1.add(kb.layers.Dense(7, input_shape = [9], activation = "relu"))
my_model1.add(kb.layers.Dense(3, activation = "relu"))
my_model1.add(kb.layers.Dense(1, activation = "sigmoid"))

my_model1.summary()




Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 7)                 70        
                                                                 
 dense_61 (Dense)            (None, 3)                 24        
                                                                 
 dense_62 (Dense)            (None, 1)                 4         
                                                                 
Total params: 98
Trainable params: 98
Non-trainable params: 0
_________________________________________________________________


In [21]:
# 2. Functional API

inputs = kb.Input(shape = (9,))

x = kb.layers.Dense(7, activation = tf.nn.relu)(inputs)

x = kb.layers.Dense(3, activation = tf.nn.relu)(x)

# stuff
outputs = kb.layers.Dense(1, activation = tf.nn.sigmoid)(x)

my_model2 = kb.Model(inputs = inputs, outputs = outputs)


# Functional API building

my_model2.summary()


Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 9)]               0         
                                                                 
 dense_57 (Dense)            (None, 7)                 70        
                                                                 
 dense_58 (Dense)            (None, 3)                 24        
                                                                 
 dense_59 (Dense)            (None, 1)                 4         
                                                                 
Total params: 98
Trainable params: 98
Non-trainable params: 0
_________________________________________________________________


In [16]:
# 3. Subclass

class MyModel(kb.Model):
    def __init__(self):
        super().__init__()
        self.layer2 = kb.layers.Dense(7, input_shape = [9], activation = "relu")
        self.layer3 = kb.layers.Dense(3, activation = "relu")
        self.layer4 = kb.layers.Dense(1, activation = "sigmoid")


    def call(self, inputs):
        x = self.layer2(inputs)
        x = self.layer3(x)
        return(self.layer4(x))

# define input shape
inputs = kb.Input(shape = (9,))

# create a model using your custom class
x = MyModel()(inputs)

# shove the inputs and outputs into a model object
my_model = kb.Model(inputs = inputs, outputs = x)

# show me the model
my_model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 9)]               0         
                                                                 
 my_model_5 (MyModel)        (None, 1)                 98        
                                                                 
Total params: 98
Trainable params: 98
Non-trainable params: 0
_________________________________________________________________


## Functions

We've already used all these functions, but as a quick refresher:

- `.summary()`: call it on a model to see the structure of the model as well as information about they layers
- `.compile()`: tells python *how* to train your model, e.g. which optimizer to use, which metrics to collect, what your test/validation set is. 
- `.fit()`: train your model given the data (input and output), number of `epochs`, etc (just like sklearn but with more options)
- `.predict()`: use your model to make predictions given some input values (just like sklearn)



If you have time, download [this data set](https://www.kaggle.com/datasets/chaunguynnghunh/sepsis?select=Paitients_Files_Train.csv) and train one or all of the models you built on it (Don't include `ID` as a predictor). Don't forget to z-score and to use [`LabelBinarizer()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html) to change the outputs to 0's and 1's. Use whatever optimizer you want. 